<!-- Simon-Style -->
<p style="font-size:19px; text-align:left; margin-top:    15px;"><i>German Association of Actuaries (DAV) — Working Group "Explainable Artificial Intelligence"</i></p>
<p style="font-size:25px; text-align:left; margin-bottom: 15px"><b>Use Case SOA GLTD Experience Study:<br>
USE CASE GLTD - H-statistic with dependent categorical inputs
</b></p>
<p style="font-size:19px; text-align:left; margin-bottom: 15px; margin-bottom: 15px">Guido Grützner (<a href="mailto:guido.gruetzner@quantakt.com">guido.gruetzner@quantakt.com</a>)

# Introduction

The identification of interactions is an important goal when trying to understand models which result from a supervised learning task. One standard approach to do this is the H-statistic [see e.g. [Molnar]](https://christophm.github.io/interpretable-ml-book/interaction.html#theory-friedmans-h-statistic). In this notebook we will 
* explain why the H-statistics makes sense under the assumption of independent inputs,
* and propose an alternative approach, the correlated H-statistic, for dependent, categorical inputs.

We will show and explain that this alternative has the following properties
* The correlated H-statistic only uses the actual observed joint probabilities, not probabilities from independent margins.
* It not only works for fitted models but can also provide information on the raw data.
* It is very fast to compute, even for a sizeable number of inputs, which may have a large number of categories.

Together with theoretical considerations, this notebook provides a (didactic) reference implementation by way of a simple example from the SOA GLTD dataset. This is the dataset which is used in all notebooks of the Use Case GLTD. Further background on this dataset is available in the other notebooks of this Use Case and the original SOA experience study (see [2019 Group Long-Term Disability Experience Study Preliminary Report](https://www.soa.org/4a7e84/globalassets/assets/files/resources/experience-studies/2019/2019-gltd-study-report.pdf)). 

# Definition and background for the H-statistic

The H-statistic is normally introduced by its sampling estimator. For the sample $(a_k,b_k)_{k=1}^N$ from an independent pair of random variables $(A,B)$ and a zero mean, i.e. centred function $f$:
$$ H=\frac{\sum_{k=1}^N \big(f(a_k,b_k) - f_A(a_k) - f_B(b_k)\big)^2}{\sum_{k=1}^N \big(f(a_k,b_k)\big)^2}$$
where $f_A$ and $f_B$ are the partial dependence functions of $f$, i.e. $f_A(a) = \frac 1 N \sum_{k=1}^N f(a,b_k)$ and $f_B(b) = \frac 1 N \sum_{k=1}^N f(a_k,b).$ 

But the motivation behind the H-statistic is best understood in the broader context of the functional ANOVA decomposition. If a function $f$ of an independent pair of random variables $(A,B)$ has finite variance, then the random variable $f(A,B)$ allows the decomposition:
$$ f(a,b) = f_0 + f_A(a) + f_B(b) + f_{A,B}(a,b),$$
where 
* $f_0 = \mathbb{E}_{A,B}[f(A,B)]$ is a constant function.
* $f_A(a) = \mathbb{E}_B[f(a,B)] - f_0$ and $f_B(b) = \mathbb{E}_A[f(A,b)] - f_0$ are the partial dependence in particular univariate, functions,
* and $f_{A,B}(a,b) = f(a,b) - f_0 - f_A(a) - f_B(b)$ is a bivariate function.

This decomposition has some appealing properties
1. it is unique,
2. all summands are uncorrelated,
3. the sum $f_0 + f_A(a) + f_B(b)$ is the best approximation to $f$ by additive functions in the least squares (i.e. ${L}^2$ ) sense.

From 2. above it is clear that the variance $\mathbb{V}[f]$ allows the decomposition 
$$ \mathbb{V}[f] = \mathbb{V}[f_A] + \mathbb{V}[f_B] + \mathbb{V}[f_{A,B}]$$
and 3. motivates the characterisation of $f_{A,B}$ as the "pure interaction term".  Acknowledging these two facts, the H-statistic is simply 
$$ H=\frac{\mathbb{V}[f_{A,B}]}{\mathbb{V}[f]},$$
which can be interpreted as the relative amount of variance explained by the pure interaction term. Here, and in this whole report, all stochastic quantities are always understood with respect to the empirical probability distribution defined by the observations. In particular, we do not distinguish between population quantities and sample estimates.

The functional ANOVA decomposition works in the same way for multivariate inputs. Of course, in the multivariate case, there will be one univariate component for each input, an interaction term for each pair of inputs and higher order interactions, i.e. with three, four, ..., inputs as well. But, for the definition of the H-statistic, only pairs of inputs and the according bivariate decompositions are needed.

A discussion of functional ANOVA can be found in [Saltelli](http://www.andreasaltelli.eu/file/repository/PUBLISHED_PAPER.pdf), while [Kuo et.al.](https://www.ams.org/journals/mcom/2010-79-270/S0025-5718-09-02319-9/S0025-5718-09-02319-9.pdf) provide detailed proofs in a Hilbert-Space setting.

## Problems with the H-statistic

The H-statistic has two problems, one fundamental and one practical. The fundamental issue is the assumption of independence, which is rarely met for real data. A calculation based on independence distorts the true probabilities and leads to impossible data issues. For a case study and more explanations on this topic, see the notebook "Case_Study_depPDP". The practical issue is caused by the estimation based on samples. For one pair of inputs $O(N^2)$ function evaluations are required, which means that the calculation of all H-statistics for the interactions of $d$ inputs requires the order of $O(d^2N^2)$ function evaluations.

The proposed remedy is somewhat obvious: Just use the true probabilities and conditional expectations instead of partial dependence functions. This avoids the distortion of the data generation process and avoids impossible data. The use of conditional expectations is possible, since all inputs are categorical. In this case, the calculation of conditional expectations is straightforward and can be implemented in a highly optimized manner using just a single evaluation of the model on the data. Furthermore, since no impossible data is involved, the statistics can be calculated already on the raw data, without requiring any model at all.

But, if inputs are dependent, the functional ANOVA decomposition is no longer available. While one can still define the components, as above, just using conditional expectations, they will no longer be uncorrelated, which in turn means that an additive attribution of the variances to single components is no longer possible. But it is possible to carefully work around this problem, and come up with a weaker decomposition, which, nevertheless, is just sufficient to define a correlated H-statistic.       

## The basic concepts and their implementation 

To properly define and implement the correlated H-statistic using linear algebra, we need to go from single functions, to vector spaces of functions and recognize the underlying geometry provided by the probability distribution. At the same time, we will try to demonstrate in examples how the mathematical concepts are reflected and ultimately implemented in code. For this purpose, we will rely on an example based on the GLTD use case. We use the same dataset and the same utility function to load the data as for the other notebooks.

### Initialisation

In [27]:
from IPython.display import display, display_html

from scipy import linalg

import os
import sys
module_path = os.path.abspath(os.path.join(os.getcwd(), '../report versions/'))
if module_path not in sys.path:
    sys.path.append(module_path)

import gltd_utilities

import itertools
import numpy as np
import pandas as pd
pd.options.mode.copy_on_write = True

In [28]:
(X, Y, ID, nm_cat, nm_num, seed, rng) = gltd_utilities.load_gltd_data(
                                            "d:/tmp/GLTD data/", pct=1)
for vnm in nm_cat:
    X[vnm] = X[vnm].cat.remove_unused_categories()
seed

'197746953433573068013957040465879210157'

### Function spaces

Since all inputs are categorical, the space of functions of these categorical variables is a finite dimensional vector spaces. We demonstrate first how these functions can be understood and implemented as Pandas series with multi-indices. This is done using an extract consisting of just two input variables from the GLTD dataset, which we will use throughout this notebook.

In [29]:
# load a pair of inputs and observations from the data 
demodata = pd.concat([X[["Diagnosis_Category", "Gender"]], Y], axis=1)
# Simplify Diagnosis_Category because there are a lot of categories     
demodata["Diagnosis_Category"] = demodata["Diagnosis_Category"].map(
    lambda cat: cat if cat in ["Circulatory", "Maternity"] else "other").astype("category")
print(f"There are {len(demodata)} observations in the data.")
print(demodata.dtypes)
print(f"The variable `Diagnosis_Category` has the categories {list(demodata["Diagnosis_Category"].cat.categories)}")
print(f"The variable `Gender` has the categories {list(demodata["Gender"].cat.categories)}")
print(f"The variable `Actual_Recoveries` takes the values {list(demodata["Actual_Recoveries"].unique())}")

There are 6388739 observations in the data.
Diagnosis_Category    category
Gender                category
Actual_Recoveries        int64
dtype: object
The variable `Diagnosis_Category` has the categories ['Circulatory', 'Maternity', 'other']
The variable `Gender` has the categories ['F', 'M']
The variable `Actual_Recoveries` takes the values [0, 1]


In [30]:
# group the data to create a multi-index with unique 2-tuples
grouped_df = demodata.groupby(
    ["Diagnosis_Category", "Gender"], 
    observed=True)["Actual_Recoveries"].agg(
    f = "mean", n = "count")
idx_2 = grouped_df.index
print(idx_2.is_unique)
f = grouped_df["f"]
display(f.to_frame())
print(f"Dimension of the space is: {len(idx_2)}")

True


f
Diagnosis_Category Gender          
Circulatory        F       0.007608
                   M       0.007293
Maternity          F       0.258492
other              F       0.015333
                   M       0.012845

Dimension of the space is: 5


In the example above, $f$ represents a function on the multi-index `idx_2` with unique tuples defined by `Diagnosis_Category` and `Gender`. Every 2-tuple from the two categories can take on any real value. This means that the dimension of the function space is exactly the number of unique tuples or - in Pandas terms - it is the length of the index. But, note that the index does not contain the tuple `(Maternity, M)`. None of the 6.4 million observations had this combination, for obvious, biological reasons. This is an example of impossible data, and the domain of definition is not a product. Hence, the inputs can never be independent. Knowing that the `Diagnosis_Category` is `Maternity`, tells you immediately something about `Gender`. Nevertheless, the interpretation as a space of functions and the calculation of dimension is not affected by this at all.

To conclude:
* A function with discrete domain is implemented as a Pandas series on a (multi-)index with unique tuples.
* The dimension of the function space over this domain is the length of this index.

Those functions which take on the value 1 for a single tuple of the index and 0 for all other tuples form a basis for this function space. This basis will be called the tuple-basis. For the dataset above, this would be a function which is 1 if the input is the tuple `(Circulatory, F)` and zero for all other inputs, another basis functions would be non-zero only for the input `(Circulatory, M)` and so on. In total 5 basis functions for the 5 possible tuples as input, which can replicate the values of every function defined on those 5 tuples by appropriate linear combinations.        

In [31]:
# create the tuple basis
tuple_basis = pd.DataFrame(np.identity(len(idx_2)), index=idx_2)
# this just for the joint display
styled_A = tuple_basis.style.set_table_attributes("style='display:inline'")
styled_A.set_caption("The tuple basis")
display_html(styled_A._repr_html_(), raw=True)

The tuple basis 
 
 
   
   
 0 
 1 
 2 
 3 
 4 
 
 
 Diagnosis_Category 
 Gender 
   
   
   
   
   
 
 
 
 
 Circulatory 
 F 
 1.000000 
 0.000000 
 0.000000 
 0.000000 
 0.000000 
 
 
 M 
 0.000000 
 1.000000 
 0.000000 
 0.000000 
 0.000000 
 
 
 Maternity 
 F 
 0.000000 
 0.000000 
 1.000000 
 0.000000 
 0.000000 
 
 
 other 
 F 
 0.000000 
 0.000000 
 0.000000 
 1.000000 
 0.000000 
 
 
 M 
 0.000000 
 0.000000 
 0.000000 
 0.000000 
 1.000000

As a next step, we need to introduce the probability measure $p_2$. In the grouping operation, the number of occurrences of the tuples in the original data was counted. Normalizing the total count to one, i.e. dividing by the total number of observations, defines the probability measure, i.e. the joint probability distribution of the tuples, respectively of the two inputs. The measure $p_2$ is defined on the same index as the functions, since there is one probability for each tuple. 

In [32]:
p_2 = grouped_df["n"] / grouped_df["n"].sum()
p_2.name = "p_2"
p_2.to_frame()

p_2
Diagnosis_Category Gender          
Circulatory        F       0.055652
                   M       0.079041
Maternity          F       0.003617
other              F       0.487749
                   M       0.373941

Conclusion: A probability measure on a discrete space can also be implemented as a Pandas series on an index. Of course, in contrast to a general function, the values of the series have to be positive and sum to one over the index. Note, that we assume probabilities to be positive, i.e. we do not want unobserved/impossible combinations in the index. For this reason, the groupings above were always done with the option `observed=True`.    

### Conditional distributions and expectations 

Conditioning means replacing the index by a coarser one. In the example, going from an index with two levels, for the two inputs, to an index with a single level. Since the coarser index can no longer distinguish function values which differ in the dropped level, these different values are replaced by the single constant average, where the average is weighted by $p_2$. Conditioning of the constant function 1 defines conditional distributions, which are each of the two marginal distributions, since in each case only a single index is left.

The coarsening, i.e. the transition from the 2-level index to the single level index, is again done by grouping followed by summing. 

In [33]:
# determine the two marginal distributions, one for ach level of the index
p_A = p_2.groupby("Diagnosis_Category", observed=True).sum()
p_A.name = "p_A"
p_B = p_2.groupby("Gender", observed=True).sum()
p_B.name = "p_B"

# this is just for the joint display
styled_A = p_A.to_frame().style.set_table_attributes("style='display:inline'")
styled_B = p_B.to_frame().style.set_table_attributes("style='display:inline'")
display_html(styled_A._repr_html_()
             + styled_B._repr_html_(), raw=True)

,p_A
Diagnosis_Category,
Circulatory,0.134693
Maternity,0.003617
other,0.861690
,p_B
Gender,
F,0.547018
M,0.452982


The conditional expectation of a non-constant function $f$ works in the same way. We only have to incorporate the $f$ values.  

In [34]:
f_weighted = f * p_2
f_A = f_weighted.groupby("Diagnosis_Category", observed=True).sum()
f_A = f_A / p_A
f_A.name = "f_A"
f_B = f_weighted.groupby("Gender", observed=True).sum()
f_B = f_B / p_B
f_B.name = "f_B"

# this is just for the joint display
styled_A = f_A.to_frame().style.set_table_attributes("style='display:inline'")
styled_B = f_B.to_frame().style.set_table_attributes("style='display:inline'")
display_html(styled_A._repr_html_()
             + styled_B._repr_html_(), raw=True)

,f_A
Diagnosis_Category,
Circulatory,0.007423
Maternity,0.258492
other,0.014253
,f_B
Gender,
F,0.016155
M,0.011876


Since conditional expectations are random variables in their own right, we can calculate their expected values. As a sanity check on the calculations this far, we calculate the expectation of $f$ in three different ways:
1. From the original demo dataset using `mean`. The Pandas function `mean` is the unweighted, arithmetic average. The according measure is the uniform measure, which reflects the iid nature of the original data.
2. From the aggregated function with unique index using the measure $p_2$, which results from aggregation of iid observations. 
3. From the two marginal distributions with the appropriate conditional expectations. 

The next block is just a quick sanity check, to make sure everything works as expected and the different ways to calculate the expected value of the `Actual_Recoveries` all agree. 

In [35]:
f_1 = demodata["Actual_Recoveries"].mean()
f_2 = (f * p_2).sum()
f_3_A = (f_A * p_A).sum()
f_3_B = (f_B * p_B).sum()

print(f_1, f_2, f_3_A, f_3_B)

0.014216733536931153 0.014216733536931155 0.014216733536931155 0.014216733536931155


## Embedding and broadcasting

Since a conditional expectation reduces the index, the resulting function is obviously defined on a different index than the original one. Just compare the Pandas series for $f_A$ or $f_B$ with the one for $f$ in the examples above. This is a problem since our ultimate goal is to split a function into components, such that the components sum up to the original function. Such a decomposition can only make sense, if all components are defined on the same domain as the original function. This means we need to find a way to embed functions on a coarse index, i.e. with few levels, back into an index with more levels. In Pandas, this can be accomplished by the broadcasting or reindexing mechanism.   

In [36]:
# broadcast f_A back to the original index
# for whatever reason this does not work for a CategoricalIndex 
# so we need a cast to "string". 
tmp = pd.Series(f_A, index=f_A.index.astype(str))
f_A_broadcast = tmp.reindex(index=idx_2, level="Diagnosis_Category")
f_A_broadcast.name = "broadcast"

# this is just for the joint display
styled_A = f_A.to_frame().style.set_table_attributes("style='display:inline'")
styled_B = f_A_broadcast.to_frame().style.set_table_attributes("style='display:inline'")
display_html(styled_A._repr_html_()
             + styled_B._repr_html_(), raw=True)

f_A 
 
 
 Diagnosis_Category 
   
 
 
 
 
 Circulatory 
 0.007423 
 
 
 Maternity 
 0.258492 
 
 
 other 
 0.014253 
 
 
 

 
 
 
   
   
 broadcast 
 
 
 Diagnosis_Category 
 Gender 
   
 
 
 
 
 Circulatory 
 F 
 0.007423 
 
 
 M 
 0.007423 
 
 
 Maternity 
 F 
 0.258492 
 
 
 other 
 F 
 0.014253 
 
 
 M 
 0.014253

This broadcasting is just the concrete implementation of the conceptional identification between a function with a single input, and a function of two inputs, which is constant with respect to one of its inputs. In the example above, the broadcasted $f_A$ is constant with respect to `Gender`. It is easy to verify, that the conditional expectation of the broadcast function with respect to `Diagnosis_Category` is again the original $f_A$.

## The inner product

Recall, that the probability measure $p_2$ defines an inner product on its space of random variables with finite variance. The inner product $\langle \cdot, \cdot\rangle$ between $h=h(A,B)$ and $g=g(A,B)$ is the expected value with respect to $p_2$ of their product, i.e.
$$\langle g, h\rangle=\mathbb{E}_{(A,B)}[g \cdot h].$$
As an immediate consequence, probabilistic notions can be directly translated into geometric ones, and vice versa. For zero mean functions $g$ and $h$ we will use  that:
* The variance of $g$ is equal to the square norm $\| g \|^2 = \langle g, g\rangle$.
* $g$ and $h$ are uncorrelated if and only if $g$ and $h$ are orthogonal $\langle g, h\rangle = 0$ or $g\perp h.$

But, before we are able to fully use numerical linear algebra from numpy or scipy, we need to resolve one remaining issue. The standard numerical routines assume coordinates, for which the inner product is the standard Euclidean one, or, equivalently, that the basis vectors are orthonormal. It is convenient to use the tuple-basis introduced above. But this basis is not orthonormal. As the following computation of the Gram matrix shows. Recall that the Gram matrix is the matrix $G$ made up of all inner products of the basis vectors: $G_{ij} = \mathbb{E}[b_i b_j]$.

In [37]:
# create the tuple basis
tuple_basis = pd.DataFrame(np.identity(len(idx_2)), index=idx_2)

# create Gram-matrix
gramdf = pd.DataFrame(0.0, index=range(5), columns=range(5))
for (ii,jj) in itertools.combinations_with_replacement(range(5),2):
    gramdf.loc[ii,jj] = (tuple_basis[ii] * tuple_basis[jj] * p_2).sum()
    gramdf.loc[jj,ii] = gramdf.loc[ii,jj]

# this just for the joint display
styled_A = tuple_basis.style.set_table_attributes("style='display:inline'")
styled_A.set_caption("Coordinates of tuple basis vectors")
styled_B = gramdf.style.set_table_attributes("style='display:inline'")
styled_B.set_caption("Gram Matrix")
display_html(styled_A._repr_html_() + styled_B._repr_html_(), raw=True)

Coordinates of tuple basis vectors 
 
 
   
   
 0 
 1 
 2 
 3 
 4 
 
 
 Diagnosis_Category 
 Gender 
   
   
   
   
   
 
 
 
 
 Circulatory 
 F 
 1.000000 
 0.000000 
 0.000000 
 0.000000 
 0.000000 
 
 
 M 
 0.000000 
 1.000000 
 0.000000 
 0.000000 
 0.000000 
 
 
 Maternity 
 F 
 0.000000 
 0.000000 
 1.000000 
 0.000000 
 0.000000 
 
 
 other 
 F 
 0.000000 
 0.000000 
 0.000000 
 1.000000 
 0.000000 
 
 
 M 
 0.000000 
 0.000000 
 0.000000 
 0.000000 
 1.000000 
 
 
 

 
 Gram Matrix 
 
 
   
 0 
 1 
 2 
 3 
 4 
 
 
 
 
 0 
 0.055652 
 0.000000 
 0.000000 
 0.000000 
 0.000000 
 
 
 1 
 0.000000 
 0.079041 
 0.000000 
 0.000000 
 0.000000 
 
 
 2 
 0.000000 
 0.000000 
 0.003617 
 0.000000 
 0.000000 
 
 
 3 
 0.000000 
 0.000000 
 0.000000 
 0.487749 
 0.000000 
 
 
 4 
 0.000000 
 0.000000 
 0.000000 
 0.000000 
 0.373941

So the tuple basis is orthogonal but not normalised. Since a basis vector $b_i$ has squared length
$$\|b_i\|^2 = \mathbb{E}[b_i b_i] = \mathbb{E}[b_i] = p_i,$$
where the fact is used that $b_i$ takes only the values 0 and 1 and, accordingly $b_i^2=b_i.$  
To obtain normalised basis vectors $\tilde{b}_i$ we need to scale them as $\tilde{b}_i = \frac{1}{\sqrt{p_i}}b_i.$ Since coordinates transform with the inverse we have the following rules:
* To transform tuple coordinates to normalised coordinates, multiply tuple coordinates by $\sqrt{p_i}$.
* To transform normalised coordinates to tuple coordinates, multiply normalised coordinates by $\frac{1}{\sqrt{p_i}}$.

After this transformation, one can use all standard linear algebra routines, as found in numpy or scipy, using the new coordinates. As an admittedly obvious example, here the calculation of the Gram matrix again, but this time only using numpy.   

In [38]:
# transform tuple basis to normalised basis
norm_basis = tuple_basis.mul(np.sqrt(p_2), axis=0)
# do plain vanilla numpy matrix computation
tmpmat = norm_basis.to_numpy()
tmpmat @ tmpmat.T

array([[0.05565198, 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.07904095, 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.00361746, 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.48774883, 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.37394077]])

# Definition of the correlated H-statistics

Now define $\mathscr{V}_2$ as the space of all random variables, which can be written as bivariate functions with inputs $(A,B)$. The goal now is to explicitely determine or calculate the subspaces 
* $\mathscr{V}_0$: the space of constant functions
* $\mathscr{V}_1$: a space spanned by all univariate functions 
* and $V_\text{inter}$ a space of functions with interactions 
such that
$$ \mathscr{V}_2=\mathscr{V}_0 \boxplus \mathscr{V}_1 \boxplus \mathscr{V_\text{inter}},$$
where the operator $\boxplus$ means direct, orthogonal sum. This decomposition allows in turn to write any function $f_2$ from $\mathscr{V}_2$ as 
$$f_2=f_0 + f_1 + f_\mathrm{{inter}}$$
with the properties
1. The summands $f_0$, $f_1$ and $f_\text{inter}$ are uniquely defined
2. The summands $f_1$ and $f_\text{inter} = f_2 - f_0 - f_1$ are uncorrelated.
3. $f_1$ is the best approximation of $f_2$ in least squares which can be written as a linear combination of univariate functions.

These three properties are exactly the properties needed from the original FANOVA decomposition to motivate the definition of the H-statistic. This means we can define the H-statistic in terms of the decomposition above in the same way as before, i.e. as
$$ H=\frac{\mathbb{V}[f_2 - f_0 - f_1]}{\mathbb{V}[f_2]}.$$

The only difference to the independent case is, that we cannot further decompose $\mathscr{V}_1$ into orthogonal subspaces of univariate functions. Since $A$ and $B$ are dependent, those subspaces will not be orthogonal. But this is not really necessary to define the H-statistic.

The only things left to do before we can compute the correlated H-statistic are the definitions of $\mathscr{V}_1$ and $\mathscr{V}_\text{inter}.$ 

## Definition of $\mathscr{V}_1$

The space $\mathscr{V}_1$ will be defined by providing a set of basis vectors. We start with the tuple-basis for the spaces defined on input $A$, called $\mathscr{V}_A$ and input $B$, $\mathscr{V}_B$:

In [39]:
tuple_basis_A = pd.DataFrame(np.identity(len(f_A.index)), index=f_A.index.astype(str))
tuple_basis_B = pd.DataFrame(np.identity(len(f_B.index)), index=f_B.index.astype(str))

# this is just for the joint display
styled_A = tuple_basis_A.style.set_table_attributes("style='display:inline'")
styled_B = tuple_basis_B.style.set_table_attributes("style='display:inline'")
display_html(styled_A._repr_html_() + styled_B._repr_html_(), raw=True)

0 
 1 
 2 
 
 
 Diagnosis_Category 
   
   
   
 
 
 
 
 Circulatory 
 1.000000 
 0.000000 
 0.000000 
 
 
 Maternity 
 0.000000 
 1.000000 
 0.000000 
 
 
 other 
 0.000000 
 0.000000 
 1.000000 
 
 
 

 
 
 
   
 0 
 1 
 
 
 Gender 
   
   
 
 
 
 
 F 
 1.000000 
 0.000000 
 
 
 M 
 0.000000 
 1.000000

These two bases are now broadcast to $\mathscr{V}_2$, but there is a catch:

In [40]:
tuple_basis_1 = pd.concat([tuple_basis_A.reindex(index=idx_2, level=0), tuple_basis_B.reindex(index=idx_2, level=1)],
                          keys=["Diagnosis_Category", "Gender"],axis=1)
tuple_basis_1

Diagnosis_Category           Gender     
                                           0    1    2      0    1
Diagnosis_Category Gender                                         
Circulatory        F                     1.0  0.0  0.0    1.0  0.0
                   M                     1.0  0.0  0.0    0.0  1.0
Maternity          F                     0.0  1.0  0.0    1.0  0.0
other              F                     0.0  0.0  1.0    1.0  0.0
                   M                     0.0  0.0  1.0    0.0  1.0

After broadcasting, the vectors are no longer linearly independent. The sum of the first three is the constant (all ones) vector, and the sum of the last two is constant again. Furthermore, we need to treat the constant vector separately, since the space of constants shall get its own space, $\mathscr{V}_0$. This is just a manifestation of the "drop 1" requirement for one hot encoding. So we need to drop on vector from each of tuple_basis_A and _B. Here we implement "drop first".

In [41]:
tuple_basis_A = pd.DataFrame(np.identity(len(f_A.index))[:,1:], index=f_A.index.astype(str))
tuple_basis_B = pd.DataFrame(np.identity(len(f_B.index))[:,1:], index=f_B.index.astype(str))
const = pd.Series(1, index=idx_2)
tuple_basis_01 = pd.concat([const, tuple_basis_A.reindex(index=idx_2, level=0), tuple_basis_B.reindex(index=idx_2, level=1)],
                          keys=["const", "Diagnosis_Category", "Gender"], axis=1)
tuple_basis_01

const Diagnosis_Category      Gender
                              0                  0    1      0
Diagnosis_Category Gender                                     
Circulatory        F          1                0.0  0.0    0.0
                   M          1                0.0  0.0    1.0
Maternity          F          1                1.0  0.0    0.0
other              F          1                0.0  1.0    0.0
                   M          1                0.0  1.0    1.0

Inspection shows that the vectors above are linearly independent, i.e. they form a basis. Together with the constant vector, we are still able to recover the dropped vectors with tuples ("Diagnosis_Category", 2) and ("Gender", 1) from above: 

In [42]:
dc_0 = (tuple_basis_01[("const", 0)] 
        - tuple_basis_01[("Diagnosis_Category", 0)] - tuple_basis_01[("Diagnosis_Category", 1)])
g_1 = (tuple_basis_01[("const", 0)] - tuple_basis_01[("Gender", 0)])
pd.concat([dc_0, g_1], axis=1)

0    1
Diagnosis_Category Gender          
Circulatory        F       1.0  1.0
                   M       1.0  0.0
Maternity          F       0.0  1.0
other              F       0.0  1.0
                   M       0.0  0.0

To properly count the dimensions of the spaces, set $\text{dim}\mathscr{V}_2 =\text{len}\, (idx_2) = n$ and define $c_A$ and $c_B$ as the number of levels of $A$ respectively $B$. Since the decomposition of the subspaces is a direct sum, their dimensions add up to the total dimension:
$$ \text{dim}\mathscr{V}_2 = \text{dim}\mathscr{V}_0 + \text{dim}\mathscr{V}_1 + \text{dim}\mathscr{V}_\text{inter}.$$
By construction of the index, $ n\geq 1$ and $\text{dim}\,\mathscr{V}_0=1$. If the number of observed tuples is large enough, i.e. if $n-1 \geq c_A + c_B - 2$ then 
$$ \text{dim}\mathscr{V}_1 = c_A + c_B - 2 \quad\text{ and }\quad \text{dim}\mathscr{V}_\text{inter} = n - 1 - (c_A + c_B - 2).$$
If the index becomes smaller, interactions are not possible. If $n - 1 \leq (c_A + c_B - 2)$ then
$$ \text{dim}\mathscr{V}_1 = n-1 \quad\text{ and }\quad \text{dim}\mathscr{V}_\text{inter} = 0.$$ 
In our example $n=5$, $c_A = 3$ and $c_B=2$, so that $c_A + c_B - 2 = 3 \leq 4 = n - 1$ and we can conclude
$$ \text{dim}\mathscr{V}_1 = 3 \quad\text{ and }\quad \text{dim}\mathscr{V}_\text{inter} = 1.$$

When constructing the basis of $\mathscr{V}_1$ in practice, it does not matter how we choose the basis vectors. If the number is OK and if they are linearly independent, we are fine. Here, we just take the joint basis for $\mathscr{V}_0 + \mathscr{V}_1$ from above.

In [43]:
tuple_basis_01

const Diagnosis_Category      Gender
                              0                  0    1      0
Diagnosis_Category Gender                                     
Circulatory        F          1                0.0  0.0    0.0
                   M          1                0.0  0.0    1.0
Maternity          F          1                1.0  0.0    0.0
other              F          1                0.0  1.0    0.0
                   M          1                0.0  1.0    1.0

## Construction of $\mathscr{V}_\text{Inter}$

A fast and numerically stable way to construct the orthogonal complement is the full QR-decomposition, as provided by the function `scipy.linalg.qr`. As mentioned above, we need to perform a change of coordinates before, and after application of the decomposition.

In [44]:
# transformation of normalised basis and extraction of the numpy array
A = tuple_basis_01.multiply(np.sqrt(p_2), axis=0).to_numpy()
Q, R = linalg.qr(A)
Q.shape

(5, 5)

By definition of the QR decomposition, $Q$ is an orthonormal basis of $\mathscr{V}_2$ where the first four vectors are a basis for the column space of $A$, which is $\mathscr{V}_0 + \mathscr{V}_1$ and the last one a basis for the orthogonal complement, i.e. $\mathscr{V}_\text{Inter}.$ Of course, this is in normalised coordinates, so we have to transform this vector back.

In [45]:
tuple_basis_inter = pd.DataFrame(Q[:,-1:], index=idx_2, columns=["Interaction"]).multiply(1 / np.sqrt(p_2), axis=0)
tuple_basis_inter

Interaction
Diagnosis_Category Gender              
Circulatory        F      -3.022426e+00
                   M       2.128062e+00
Maternity          F       2.460586e-15
other              F       3.448579e-01
                   M      -4.498146e-01

It is a good thing to verify the orthogonality of the subspaces. The following calculation shows $\mathscr{V}_0 + \mathscr{V}_1\perp $\mathscr{V}_\text{Inter}$. The result is shown in full machine precision, i.e. without rounding. This is just to remind the reader, that operations on large matrices may create numerical issues. But in our toy example the results are reassuringly "orthogonal", i.e. zero up to machine precision. 

In [46]:
for col in tuple_basis_01.columns:
    print((tuple_basis_01[col] * tuple_basis_inter["Interaction"] * p_2).sum())

-1.1102230246251565e-16
8.901069979099821e-18
0.0
-8.326672684688674e-17


We have not explicitely 

# Calculation of the statistics

Any function defined on the index can no be decomposed by expressing it in terms of the basis. This can be done using $Q$ in normalised coordinates or using the transformed vectors in the tuple-coordinates. It is slightly more efficient to do this directly in numpy using normalised coordinates. Here we decompose the raw data vector.

In [ ]:
# transform f to normalised coordinates
fnc = f * np.sqrt(p_2)
# find coefficients wrt to basis Q
x = linalg.solve(Q,fnc)

The coordinates relate directly to the subspaces. The coefficient of $R$ ensures the correct sign of the mean.

In [ ]:
dim_bas = tuple_basis_01.shape[1]
f_mean = R[0,0] * x[0]
# Variance of f
V = linalg.norm(x[1:])**2
V_1 = linalg.norm(x[1:dim_bas])**2
V_inter = linalg.norm(x[dim_bas:])**2
Hcorr = V_inter / V
print(f"Total variance is {np.round(V,6)}\n\
      thereof {np.round(V_1 / V *100,1)}% additive and {np.round(V_inter/V*100,1)}% interaction ")

Total variance is 0.000223
      thereof 99.9% additive and 0.1% interaction 


Again, one can check correctness by comparing some results.

In [ ]:
f_mean_alt = (f * p_2).sum()
print(abs(f_mean - f_mean_alt) < 1e-15)
V_alt = (f**2 * p_2).sum() - f_mean_alt**2
print(abs(V - V_alt) < 1e-15)

True
True


If the basis and the embedding space are very high dimensional, the explicit computation and storage of a $(n,n)$ matrix is no longer  feasible. In that case, the least squares solution to the equation $Ax=f$ with $A$ the normalized tuple basis and $f$ the normalized function, is a very efficient alternative. This function is provided in `scipy.linalg` as `lstsq`. 

# Scaling, run time and memory considerations

Since this example is quite limited in scope, it remains a fair question how this approach scales to real world problem sizes. The brief answer is: Very well indeed. To support this statement, note that there are only two drivers of computational resources.

The first driver is the sheer number of pairs in the input, which scale as $d^2$. Second is the dimension of the respective spaces. But both can be handled efficiently, as the worked example in "exp_anova", with 21 inputs, and some of them with high cardinality, shows.  

The size of the dataset just matters for the fitting of the model, and the prediction, i.e. the construction of $f$. After this is done, data will be aggregated to observed tuples. This is why the total size of the data (in our case roughly 6.4 million observations) does not matter, or does matter only very slightly as this aggregation is done in Pandas very efficiently.